In [1]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData, OpenCDP
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, get_svd_encoder
from src.embeddings import RecsysEmbedding

experiment_name = 'SlatewiseAttention'
device = 'cuda:0'
seed = 123
pkl_path = '../pkl/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
torch.__version__

'1.12.1'

# Модель

In [3]:
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)

class SlatewiseAttention(torch.nn.Module):
    """
    No recurrent dependency, just slate-wise attention.
    """
    def __init__(self, embedding, nheads=2, output_dim=1):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.attention= torch.nn.MultiheadAttention(
            2 * embedding.embedding_dim,
            num_heads=nheads,
            batch_first=True
        )
        
        self.out_layer = torch.nn.Linear(2 * embedding.embedding_dim, output_dim)

    def forward(self, batch):
        item_embs, user_embs = self.embedding(batch)
        shp = item_embs.shape
        
        # let model attent to first padd token if slate is empty to avoid NaN gradients
        # (anyway they does not contrinute into metrics computation)
        key_padding_mask = batch['slates_mask'].clone()
        key_padding_mask[:,:, 0] = True 
        
        # repeating user embedding for each item embeding
        features = torch.cat(
            [
                item_embs,
                user_embs[:, :, None, :].repeat(1, 1, shp[-2], 1).reshape(shp)
            ],
            dim = -1
        )
        
        # all recomendations in session are scored independently
        features = features.flatten(0,1)
        
        features, attn_map = self.attention(
            features, features, features,
            key_padding_mask=~key_padding_mask.flatten(0, 1)
        )
        
        # transforming back to sequence shape
        shp = list(shp)
        shp[-1] *= 2
        return self.out_layer(features.reshape(shp)).squeeze(-1)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [4]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = SlatewiseAttention(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
    output_dim=1
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device=device, lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


biulding affinity matrix...


3it [00:00, 4059.00it/s]


Test before learning: {'f1': 0.0, 'roc-auc': 0.6666666269302368, 'accuracy': 0.75}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | treshold: 0.49
Test: accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | 
Val update: epoch: 4 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.51
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(SlatewiseAttention(
   (embedding): RecsysEmbedding(
     (item_embeddings): Embedding(5, 32)
   )
   (attention): MultiheadAttention(
     (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
   )
   (out_layer): Linear(in_features=64, out_features=1, bias=True)
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [5]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 108 batches


In [6]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    
    model = SlatewiseAttention(
        RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings),
        output_dim=1
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating SlatewiseAttention with svd embeddings
Test before learning: {'f1': 0.0, 'roc-auc': 0.49343323707580566, 'accuracy': 0.8981701731681824}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.09749498963356018 | f1: 0.17766821384429932 | auc: 0.508638858795166 | treshold: 0.01
Test: accuracy: 0.10182981193065643 | f1: 0.18483763933181763 | auc: 0.500806450843811 | 
Val update: epoch: 1 |accuracy: 0.09749498963356018 | f1: 0.17766821384429932 | auc: 0.5540139675140381 | treshold: 0.03
Test: accuracy: 0.10182981193065643 | f1: 0.18483763933181763 | auc: 0.5496248006820679 | 
Val update: epoch: 2 |accuracy: 0.15717148780822754 | f1: 0.18285778164863586 | auc: 0.5617015361785889 | treshold: 0.04
Test: accuracy: 0.14644573628902435 | f1: 0.18896299600601196 | auc: 0.5634030699729919 | 
Val update: epoch: 3 |accuracy: 0.8416833877563477 | f1: 0.2046598643064499 | auc: 0.5743207931518555 | treshold: 0.09
Test: accuracy: 0.8348771333694458 | f1: 0.21209490299224854 | auc: 0.5712577104568481 | 
Val update: epoch: 4 |accuracy: 0.10552533715963364 | f1: 0.17878358066082 | auc: 0.5939933061599731 | treshold: 0.060000000000000005
Test: accuracy: 0.10695

train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.09743773192167282 | f1: 0.17757314443588257 | auc: 0.6103076338768005 | treshold: 0.02
Test: accuracy: 0.1018146499991417 | f1: 0.18481266498565674 | auc: 0.6058329343795776 | 
Val update: epoch: 1 |accuracy: 0.8380045890808105 | f1: 0.21097399294376373 | auc: 0.6292380094528198 | treshold: 0.17
Test: accuracy: 0.8373027443885803 | f1: 0.21538236737251282 | auc: 0.6310606598854065 | 
Val update: epoch: 2 |accuracy: 0.8716433048248291 | f1: 0.2133520543575287 | auc: 0.6635662317276001 | treshold: 0.17
Test: accuracy: 0.8688204288482666 | f1: 0.22498880326747894 | auc: 0.664330780506134 | 
Val update: epoch: 3 |accuracy: 0.7721443176269531 | f1: 0.2541467547416687 | auc: 0.6678265929222107 | treshold: 0.16
Test: accuracy: 0.7692039608955383 | f1: 0.25445640087127686 | auc: 0.6663146018981934 | 
Val update: epoch: 4 |accuracy: 0.722602367401123 | f1: 0.26223018765449524 | auc: 0.6845580339431763 | treshold: 0.16
Test: accuracy: 0.7113230228424072 | f1: 0.

In [7]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [8]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [9]:
for embeddings in ['neural','explicit', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = SlatewiseAttention(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings,
            embedding_dim=40
        ),
        output_dim=1
    ).to(device)

    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)


Evaluating SlatewiseAttention with neural embeddings
Test before learning: {'f1': 0.3586075007915497, 'roc-auc': 0.5385457277297974, 'accuracy': 0.4319671094417572}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.7748512625694275 | f1: 0.8341144323348999 | auc: 0.8506377935409546 | treshold: 0.45
Test: accuracy: 0.7719018459320068 | f1: 0.8338704705238342 | auc: 0.8458254933357239 | 
Val update: epoch: 1 |accuracy: 0.8123881220817566 | f1: 0.8625108599662781 | auc: 0.8940484523773193 | treshold: 0.45
Test: accuracy: 0.8066255450248718 | f1: 0.8591408133506775 | auc: 0.8882343173027039 | 
Val update: epoch: 2 |accuracy: 0.8238765597343445 | f1: 0.869563639163971 | auc: 0.907374382019043 | treshold: 0.47000000000000003
Test: accuracy: 0.8190545439720154 | f1: 0.8666346073150635 | auc: 0.9022156000137329 | 
Val update: epoch: 3 |accuracy: 0.8354858756065369 | f1: 0.8739950060844421 | auc: 0.9128892421722412 | treshold: 0.5
Test: accuracy: 0.8310965895652771 | f1: 0.8711468577384949 | auc: 0.9083013534545898 | 
Val update: epoch: 4 |accuracy: 0.8396459221839905 | f1: 0.8755583763122559 | auc: 0.9171074628829956 | treshold: 0.49
Test: accuracy: 0.8348204493522644 | 

train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.7998839020729065 | f1: 0.8491742610931396 | auc: 0.8775261044502258 | treshold: 0.46
Test: accuracy: 0.8058517575263977 | f1: 0.8541905283927917 | auc: 0.8797686100006104 | 
Val update: epoch: 1 |accuracy: 0.8115174174308777 | f1: 0.8545078039169312 | auc: 0.887147843837738 | treshold: 0.41000000000000003
Test: accuracy: 0.814339280128479 | f1: 0.8571109771728516 | auc: 0.8887879848480225 | 
Val update: epoch: 2 |accuracy: 0.8090988397598267 | f1: 0.864112913608551 | auc: 0.8989317417144775 | treshold: 0.46
Test: accuracy: 0.8097690939903259 | f1: 0.8650670051574707 | auc: 0.8982847929000854 | 
Val update: epoch: 3 |accuracy: 0.7893871068954468 | f1: 0.8545709848403931 | auc: 0.9001450538635254 | treshold: 0.45
Test: accuracy: 0.7922137379646301 | f1: 0.8568906784057617 | auc: 0.9008466005325317 | 
Val update: epoch: 4 |accuracy: 0.8018913269042969 | f1: 0.8612188696861267 | auc: 0.9069548845291138 | treshold: 0.48000000000000004
Test: accuracy: 0.8040

/opt/conda/envs/myenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


Val update: epoch: 8 |accuracy: 0.8283509612083435 | f1: 0.874733030796051 | auc: 0.9132624864578247 | treshold: 0.46
Test: accuracy: 0.8294764757156372 | f1: 0.8757488131523132 | auc: 0.9107435941696167 | 
Val update: epoch: 9 |accuracy: 0.834083080291748 | f1: 0.8772457242012024 | auc: 0.9142478704452515 | treshold: 0.49
Test: accuracy: 0.8311933279037476 | f1: 0.8755903244018555 | auc: 0.9127137660980225 | 
Val update: epoch: 11 |accuracy: 0.837348222732544 | f1: 0.8747788667678833 | auc: 0.9154505729675293 | treshold: 0.47000000000000003
Test: accuracy: 0.8374078273773193 | f1: 0.8750743269920349 | auc: 0.9135507941246033 | 
Val update: epoch: 12 |accuracy: 0.837154746055603 | f1: 0.8781070709228516 | auc: 0.9163272380828857 | treshold: 0.47000000000000003
Test: accuracy: 0.8364164233207703 | f1: 0.877951979637146 | auc: 0.9149311780929565 | 
Val update: epoch: 13 |accuracy: 0.8375658988952637 | f1: 0.8778420686721802 | auc: 0.9163583517074585 | treshold: 0.49
Test: accuracy: 0.838

train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.6911672353744507 | f1: 0.8002565503120422 | auc: 0.786144495010376 | treshold: 0.39
Test: accuracy: 0.6899528503417969 | f1: 0.8001434206962585 | auc: 0.7852516174316406 | 
Val update: epoch: 1 |accuracy: 0.7183282375335693 | f1: 0.8061391115188599 | auc: 0.8017555475234985 | treshold: 0.42000000000000004
Test: accuracy: 0.719622790813446 | f1: 0.8082488775253296 | auc: 0.8004000782966614 | 
Val update: epoch: 2 |accuracy: 0.7405552864074707 | f1: 0.8141384124755859 | auc: 0.8143574595451355 | treshold: 0.42000000000000004
Test: accuracy: 0.7409261465072632 | f1: 0.815180242061615 | auc: 0.8117127418518066 | 
Val update: epoch: 3 |accuracy: 0.7528902292251587 | f1: 0.8146038055419922 | auc: 0.8234268426895142 | treshold: 0.42000000000000004
Test: accuracy: 0.7548301219940186 | f1: 0.8167838454246521 | auc: 0.8205642104148865 | 
Val update: epoch: 4 |accuracy: 0.7593479156494141 | f1: 0.8195698857307434 | auc: 0.8312172889709473 | treshold: 0.43
Test: a

In [10]:
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# OpenCDP

In [9]:
for group in ['cosmetics', 'multi']:
    for filename in os.listdir(pkl_path):    
        result = []
        if not filename.startswith(group):
            continue
        print(f"\n == {filename} ==")
        dataset = OpenCDP.load(os.path.join(pkl_path, filename))
        (
            train_loader, 
            val_loader,
            test_loader, 
            train_user_item_matrix, 
            train_num_items
        ) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=800)
    
        print(f"{len(dataset)} data points among {len(train_loader)} batches")
        for embeddings in ['neural', 'svd']:
            print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

            model = SlatewiseAttention(
                RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings),
                output_dim=1
            ).to(device)

            best_model, metrics = train(
                model, 
                train_loader, val_loader, test_loader, 
                device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
                silent=True
            )
            
            print(metrics)
            metrics['embeddings'] = embeddings
            result.append(metrics)
        pd.DataFrame(result).to_csv(f'results/{filename}_{experiment_name}.csv')
        del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items


 == cosmetics_10_8.pkl ==
115256 data points among 116 batches

Evaluating InSlateAttention with neural embeddings
Test before learning: {'f1': 0.11245217174291611, 'roc-auc': 0.4997224807739258, 'accuracy': 0.4523639678955078}


train... loss:79.81531691551208:   0%|                                                                                              | 1/5000 [24:27<2037:45:35, 1467.48s/it]

Val update: epoch: 0 |accuracy: 0.5685881972312927 | f1: 0.68681401014328 | auc: 0.592437744140625 | treshold: 0.41000000000000003
Test: accuracy: 0.5776675939559937 | f1: 0.6960759162902832 | auc: 0.5892422199249268 | 


train... loss:77.37363070249557:   0%|                                                                                                | 2/5000 [24:54<860:56:33, 620.13s/it]

Val update: epoch: 1 |accuracy: 0.5456679463386536 | f1: 0.6963627338409424 | auc: 0.6210092306137085 | treshold: 0.38
Test: accuracy: 0.5544227361679077 | f1: 0.7044934034347534 | auc: 0.6194785237312317 | 


...

train... loss:60.980944097042084:   0%|▍                                                                                               | 25/5000 [10:11<33:48:11, 24.46s/it]

Val update: epoch: 25 |accuracy: 0.529938817024231 | f1: 0.6884518265724182 | auc: 0.5834354162216187 | treshold: 0.33
Test: accuracy: 0.5306141376495361 | f1: 0.6887058019638062 | auc: 0.562440037727356 | 
{'f1': 0.6887058019638062, 'roc-auc': 0.562440037727356, 'accuracy': 0.5306141376495361}

 == cosmetics_20_1.pkl ==


93094 data points among 93 batches

Evaluating InSlateAttention with neural embeddings
Test before learning: {'f1': 0.6832995414733887, 'roc-auc': 0.5003466010093689, 'accuracy': 0.5374177098274231}


train:   0%|                                                                                                                                       | 0/5000 [14:36<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 994.00 MiB (GPU 2; 10.91 GiB total capacity; 7.20 GiB already allocated; 499.94 MiB free; 9.75 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF